In this recipe we'll explore some of the functionality that can be applied to an alignment of protein sequences. We're going to start with a multiple sequence alignment (MSA) of myoglobin sequences from 42 diverse tetrapods, and one human hemoglobin sequence. This multiple sequence alignment was generated by following the steps in the [Progressive Multiple Sequence Alignment recipe](./Progressive%20multiple%20sequence%20alignment.ipynb).

Myoglobin was the first protein sequence to have its stucture determined by x-ray crystallography. For some background on this molecule, see the [myoglobin article in the PDB *Molecule of the Month* series](http://www.rcsb.org/pdb/101/motm.do?momID=1).

**Note:** This notebook requires that you've installed the [ete2](http://etetoolkit.org/) package, which in turn requires PyQt4. The easiest way to install PyQt4 is using [conda](https://store.continuum.io/cshop/anaconda/) (``conda install pyqt``). The full version of conda is completely free. 

First, we'll prepare our environment and load our myoglobin alignment into a [scikit-bio ``Alignment`` object](http://scikit-bio.org/docs/latest/generated/skbio.alignment.Alignment.html).

In [ ]:
%matplotlib inline

from skbio import Alignment, Protein

aln = Alignment.read('data/globin-aln.fasta', constructor=Protein)

## Exploring a sequence alignment

We can next get some information about this alignment. For example, we can easily see that there are 43 sequences in this alignment, and that it's 154 characters long. By definition, there is no variance in sequence length in an alignment, hence the standard deviation of 0.00. For this reason, it's common to think of an alignment as a matrix, where rows represent sequences and columns represent positions in the sequences.

In [ ]:
aln

There are also some mechanisms in place to get more direct access to this information. For example:

In [ ]:
aln.sequence_count()

In [ ]:
aln.sequence_length()

In [ ]:
sequence_count, sequence_length, sequence_length_std = aln.distribution_stats()
print(sequence_count, sequence_length, sequence_length_std)

The sequence identifiers in this alignment contain the genus and species that each sequence is dervied from. We can see what those are by printing the ids.

In [ ]:
for e in aln.ids():
    print(e)

We can also explore some more interesting features of this alignment. For example, we can compute the positional entropy, which is the Shannon entropy (usually abbreviated as ``H``) at each position (or column) in the alignment. In this case, we're going to treat gap characters as another *character* in the amino acid alphabet, though the right way to handle gaps in alignments when computing entropy is debatable. 

To learn more about positional entropy and information theory, see the references in the ``Alignment.positional_entropies`` help text, either on the scikit-bio website [here](http://scikit-bio.org/docs/latest/generated/skbio.alignment.Alignment.position_entropies.html) or by calling  ``help(Alignment.positional_entropies)`` in this notebook.

To keep our entropy values in the range of zero to one, we're going to pass ``base=21`` here. This is the number of characters in the canonical amino acid alphabet (20) plus the gap character. 

We can get a list of the positional entropies back (here we'll just print the first ten). A high entropy value means that there is a lot of variation in the sequences at the corresponding position in the alignment, while a low entropy value means that the corresponding position in the alignment is highly conserved. An entropy of zero means that a position is perfectly conserved (i.e., it's the same in all positions in the alignment). An entropy of one means that every character in the alphabet is present in exactly equal frequency at that position in the alignment (this is uncommon in practice).

In [ ]:
entropies = aln.position_entropies(base=21, nan_on_non_standard_chars=False)
entropies[:10]

In our alignment, there are exactly 21 positions that are perfectly conserved across all sequences. This is pretty astonishing, particularly because there is a hemoglobin sequence in the alignment. This means that (based on the information we have here) these positions have not changed since the last common ancestor of the hemoglobin/myoglobin <a href="http://en.wikipedia.org/wiki/Homology_(biology)#Paralogy">paralogs</a>). 

In [ ]:
entropies.count(0)

Let's figure out what amino acids are at these highly conserved positions. To do this, we'll first get the frequency of each amino acid at each position:

In [ ]:
position_frequencies = aln.position_frequencies()
for e in position_frequencies[:10]:
    print(e)

Then we'll print out the amino acid found at a position, for all positions with zero entropy.

In [ ]:
for i in range(aln.sequence_length()):
    if entropies[i] == 0.0:
        # find the amino acid at this position in the first sequence in the alignment
        aa = aln[0][i]
        print("Position %3d is perfectly conserved, containing %s in all sequences." % (i, aa))

Finally we can visualize the entropies along the full length of the alignment to get an idea of where the more and less conserved positions are found. In general, positions that are highly conserved are thought to have specific structural or functional roles in the protein, while positions that are less conserved might have less specific roles (e.g., providing a filler between two functional domains in the linear protein backbone).

In [ ]:
from pylab import figure

fig = figure()
ax = fig.add_subplot(111)
ax.plot(range(len(entropies)), entropies, '-')

## Building a neighbor joining tree

A common reason for building a multiple sequence alignment is for making an inference about the evolutionary relationships between the organisms. We'll do this here using the neighbor joining ([``nj``](http://scikit-bio.org/docs/latest/generated/skbio.tree.nj.html)) function in scikit-bio.

The input to ``nj`` is a distance matrix, representing the pairwise distances between the sequences in the alignment. This can be obtained using the ``distances`` method, which by default will compute the Hamming distances between all pairs of sequences in the alignment:

In [ ]:
dm = aln.distances()
print(dm)

One initial observation that we can make here is that the mean of the distribution of distances between the hemoglobin sequence and all other sequences is higher than the mean of the distribution of the chimpanzee myoglobin sequences to all of the others. This is exactly what we expect since the hemoglobin sequence diverged from the myoglobin sequences longer ago.

In [ ]:
fig = figure()
ax = fig.add_subplot(111)
ax.hist([dm['hemoglobin-human'], dm['Pan.troglodytes-chimpanzee']], label=['human hemoglobin', 'chimpanzee myoglobin'])
ax.set_xlim((0,1))
ax.legend(loc='upper left')

We can now build a neighbor joining tree from this distance matrix. At this point we don't have an easy and good visualization approach in scikit-bio for phylogenetic trees, though we're working on a solution. You can track progress on that [here](https://github.com/biocore/scikit-bio/issues/531).

In the meantime, we're going to export a newick-formatted version of the tree which can be opened with a phylogenetic tree viewing program. We'll show some exported images that we obtained using [Archeoptyrx](https://sites.google.com/site/cmzmasek/home/software/archaeopteryx), which works on OS X, Windows, and Linux, but there are many popular tree viewing programs, and most support the newick file format. A popular one for OS X is [FigTree](http://tree.bio.ed.ac.uk/software/figtree/). 

First, let's build the neighbor-joining tree:

In [ ]:
from skbio.tree import nj
tree = nj(dm)

We can then write this to file in newick format:

In [ ]:
tree_fp = 'globin-aln.ph'
tree.write(tree_fp)

We currently recommend using [ete2](http://etetoolkit.org/) for tree visualization and exploration. 

In [ ]:
import ete2

In [ ]:
from ete2 import Tree, TreeStyle
ts = TreeStyle()
Tree(tree_fp).render("%%inline", tree_style=ts)

To get a better understanding of the evolutionary history, we can use outgroup rooting to place a root in the tree. When doing outgroup rooting, you want to find the tip corresponding to the most distantly related sequence in the alignment, and look at its most recent ancestral node. Because we know *a priori* that the hemoglobin sequence is the most distantly related of the sequences in our alignment (because it's not myoglobin, so by definition had to have begun diverging before all of our myoglobin sequences began diverging), you know that its first ancestral node should be the common ancestor of all sequences in the tree.

First we find the node that we want to be the new root.

In [ ]:
new_root = tree.find('hemoglobin-human').ancestors()[0]

And then we build a new tree rooted at this node, and write that to a new file.

In [ ]:
tree = tree.root_at(new_root)

You can see that this actually gives us a pretty good picture of the relationships between the organisms. For example, notice that the deepest splits are the myoglobin versus hemoglobin clades, then the separation of the reptiles, birds, and egg-laying mammals, respectively.

In [ ]:
# Right now, ete2 can't handle naming of internal nodes, so we'll rename the root node to the empty string. 
tree.name = ''
tree.write(tree_fp)
Tree(tree_fp).render("%%inline", tree_style=ts)